In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

### Arrangement des données

In [ ]:
def load(name):
    df = pd.read_json(name, lines=True).fillna(pd.NA).sort_values('time').reset_index(drop=True)
    df = df.assign(time=(df.time - df.time[0]))
    df = df.assign(step=df.time.diff())
    return df

#### Simplifie les dates

In [ ]:
delay_df = load('../metrics-delay.json')
fast_df= load('../metrics-fast.json')

In [ ]:
bins = np.arange(0 ,2e10, 3e8)

In [ ]:
delay_df.step[delay_df.step>0].hist(log=True, bins=bins)

In [ ]:
fast_df.step[fast_df.step>0].hist(log=True, bins=bins)

In [ ]:
[ fast_df[['name','time']].groupby('name').count().plot.bar(log=True),
 delay_df[['name','time']].groupby('name').count().plot.bar(log=True)]